# Exercícios em Python 6: Biblioteca padrão python, expressões regulares, csv, xml



## Preparando o ambiente

Execute o código a seguir.

In [1]:
!git clone https://gitlab.uspdigital.usp.br/thiago/curso_ai_python.git
import sys
sys.path.append('curso_ai_python')

Cloning into 'curso_ai_python'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 92 (delta 29), reused 0 (delta 0), pack-reused 33 (from 1)
Receiving objects: 100% (92/92), 28.45 KiB | 229.00 KiB/s, done.
Resolving deltas: 100% (45/45), done.


# Se o bloco acima foi executado corretamente, você deve ver o contéudo do diretório ```curso_ai_pytho`/dados```:

In [2]:
!ls -la curso_ai_python/dados/

total 32
drwxr-xr-x 2 root root  4096 Apr 24 22:59 .
drwxr-xr-x 5 root root  4096 Apr 24 22:59 ..
-rw-r--r-- 1 root root    20 Apr 24 22:59 dados01.txt
-rw-r--r-- 1 root root    50 Apr 24 22:59 dados02.txt
-rw-r--r-- 1 root root 12633 Apr 24 22:59 telefones_epusp.txt


## 6.1 Leitura de dados em CSV

O módulo ```csv``` em Python lê arquivos em formato CSV.

A função ```csv.DictReader(arquivo)``` retorna uma lista de dicionários, um para cada linha do arquivo.

Por exemplo, o código a seguir mostra o total de alunos estrangeiros no município de Araraquara.

In [3]:
import urllib.request
import csv
import io
import tempfile
import ssl

# Suprime verificação de SSL por problemas
#  com a ICP Brasil
ignorar_ssl = ssl.create_default_context()
ignorar_ssl.check_hostname = False
ignorar_ssl.verify_mode = ssl.CERT_NONE

recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3) # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso,encoding='utf-8'), delimiter=';')
total = 0
for entrada in leitorcsv:
  if entrada['MUN']=='ARARAQUARA':
    total += int(entrada[' Nº ALUNOS '])
print(total)

50


In [4]:
entrada

{'NOMEDEP': 'ESTADUAL - SE',
 'DE': 'VOTUPORANGA',
 'DISTR': 'VOTUPORANGA',
 'MUN': 'VOTUPORANGA',
 'TIPOESC': '8',
 'COD_ESC': '44167',
 'NOMESC': 'ENNY TEREZA LONGO FRACARO PROFA',
 'DS_PAIS': 'Venezuela',
 ' Nº ALUNOS ': ' 9 '}

### Exercício 6.1.1

Encontre o total de alunos provenientes do Uruguai matriculados em escolas de SP.
O campo que contém o país é o campo com nome ```DS_PAIS```.


In [12]:
import urllib.request
import csv
import io
import ssl

ignorar_ssl = ssl.create_default_context()
ignorar_ssl.check_hostname = False
ignorar_ssl.verify_mode = ssl.CERT_NONE

recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3)  # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso, encoding='utf-8'), delimiter=';')
total = 0

for entrada in leitorcsv:
    if entrada['DS_PAIS'].strip() == 'Uruguai':
        total += int(entrada[' Nº ALUNOS '])

print(total)


18


A resposta deve ser 18

### Exercício 6.1.2

Quantos *distritos* distintos de escolas existem nesta lista? O campo que contém o tipo é ```DISTR```.
Qual o tipo com a maior quantidade de alunos?

Sugestão: Crie um dicionário.


In [16]:
import urllib.request
import csv
import io
import ssl

ignorar_ssl = ssl.create_default_context()
ignorar_ssl.check_hostname = False
ignorar_ssl.verify_mode = ssl.CERT_NONE
recurso = urllib.request.urlopen("https://dados.educacao.sp.gov.br/node/2054/download", context=ignorar_ssl)
recurso.read(3)  # Retira os 3 primeiros bytes do arquivo
leitorcsv = csv.DictReader(io.TextIOWrapper(recurso, encoding='utf-8'), delimiter=';')

distritos = {}

for entrada in leitorcsv:
    distrito = entrada['DISTR']
    num_alunos = int(entrada[' Nº ALUNOS '])

    if distrito in distritos:
        distritos[distrito] += num_alunos
    else:
        distritos[distrito] = num_alunos


num_distritos = len(distritos)
distrito_maior_qtd = max(distritos, key=distritos.get)
qtd_maior_qtd = distritos[distrito_maior_qtd]

print(f"Quantidade de distritos distintos: {num_distritos}")
print(f"O distrito com a maior quantidade de alunos é {distrito_maior_qtd} com {qtd_maior_qtd} alunos.")


Quantidade de distritos distintos: 419
O distrito com a maior quantidade de alunos é VILA MARIA com 642 alunos.


A resposta deve ser 419 distritos distintos. O tipo com a maior quantidade é o ```VILA MARIA``` com 642 alunos.

## 6.2 Processamento de XML

A url ```https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarDia(dataCotacao=@dataCotacao)?@dataCotacao='04-23-2020'&$top=100&$format=xml&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao``` recupera o relatório de cotação do dólar do Banco central em formato xml e grava-o no arquivo cotacao_dolar.xml.



In [20]:
req=urllib.request.urlopen("https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2024'&@dataFinalCotacao='05-03-2024'&$top=200&$format=xml&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao")
ff = open("cotacao_dolar.xml", "wb")
ff.write(req.read())
ff.close()



O código abaixo usa xml para encontrar o valor da cotação no dia 17/03/2024

In [18]:
import xml.etree.ElementTree
raiz = xml.etree.ElementTree.parse("cotacao_dolar.xml").getroot()
for el in raiz.iter('{http://docs.oasis-open.org/odata/ns/metadata}properties'):
  hora = el.find('{http://docs.oasis-open.org/odata/ns/data}dataHoraCotacao')
  if hora.text.startswith('2024-05-02'):
    no_venda = el.find('{http://docs.oasis-open.org/odata/ns/data}cotacaoVenda')
    valor = float(no_venda.text)
    print(valor)



5.1184


Exercício:

Encontre o *menor* valor de cotação do dólar para venda (campo com etiqueta ```{http://docs.oasis-open.org/odata/ns/data}cotacaoVenda'```).

Você deve encontrar 4.8543

In [30]:
import xml.etree.ElementTree
menorValorCotacaoVenda = float('inf')

raiz = xml.etree.ElementTree.parse("cotacao_dolar.xml").getroot()
for el in raiz.iter('{http://docs.oasis-open.org/odata/ns/metadata}properties'):
  cotacaoVenda = el.find('{http://docs.oasis-open.org/odata/ns/data}cotacaoVenda')
  if cotacaoVenda is not None:
        valor = float(cotacaoVenda.text)

        if valor < menorValorCotacaoVenda:
            menorValorCotacaoVenda = valor

print(f"O menor valor de cotação do dólar para venda é: {menorValorCotacaoVenda}")

O menor valor de cotação do dólar para venda é: 4.8543


A resposta deve ser 4.8543

## 5.3  Expressões Regulares

A linguagem Python tem uma biblioteca de expressões regulares.

A função ```re.search(exp, string)``` verifica se uma string é aderente a um determinado padrão.

### Exercícios:

O arquivo em ```/content/gdrive/My Drive/cursoai_python_aula_03/dados/telefones_epusp.txt``` contém uma lista telefônica em um texto não-estruturado com telefones úteis da escola politécnica.

Eis o conteúdo de suas 10 primeiras linhas:

In [ ]:
arquivo = open("/content/curso_ai_python/dados/telefones_epusp.txt", "rt")
for i in range(10):
  print(arquivo.readline())
arquivo.close()

### 5.3.1 Conte o número de linhas com telefones

A expressão regular ```([0-9]+([\s-]+[0-9]+)*)(\s?/\s?([0-9]+([\s-]+[0-9]+)*))*$``` casa com um ou mais números de telefone *ao final de uma linha*.

Verifique usando o módulo re quantas linhas atendem este padrão.
Você deve encontrar 255 números.

In [40]:
import re

file_path = "/content/curso_ai_python/dados/telefones_epusp.txt"

pattern = re.compile(r"([0-9]+([\s-]+[0-9]+)*)(\s?/\s?([0-9]+([\s-]+[0-9]+)*))*$")

matching_lines = 0

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        if pattern.search(line.strip()):
            matching_lines += 1

print(matching_lines)

255
